In [1]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import *
import torch
import itertools
from torch.nn.utils.rnn import pad_sequence
import math
from timeit import default_timer as timer
import torchsummary 

In [2]:
class ChessDataset(torch.utils.data.Dataset):
    X: List[Tuple[str, str]]
    def __init__(self, base_path: str, split: str, category: str, transform: Callable[[Tuple[str, str]], Tuple[str, str]] =None):
        self.__data = []
        
        with open(f"{base_path}/{split}.che-eng.{category}.che", "r") as fin, open(f"{base_path}/{split}.che-eng.{category}.en", "r") as fout:
            for line_in, line_out in zip(fin, fout):
                tokens_line_in = line_in.strip()
                tokens_line_out = line_out.strip()
                if transform is not None:
                   tokens_line_in, tokens_line_out = transform((tokens_line_in, tokens_line_out))
                self.__data.append((tokens_line_in, tokens_line_out))
    
    def __len__(self) -> int:
        return len(self.__data)
    
    def __getitem__(self, idx) -> Tuple[List[str], List[str]]:
        return self.__data[idx]
    

In [3]:
# Possible idea - add all one-move chess moves

In [4]:
# Vocab&dataloader logic
BASE_PATH = "../dataset"
SPLITS = ["train", "test", "valid"]
CATEGORIES = ["0attack", "0score", "0simple", "1attack", "1score", "1simple", "2.comparitiveattack", "2.comparitivescore", "2.comparitivesimple"]

target_token_transform = get_tokenizer('spacy', language='de_core_news_sm')

UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

train_datasets = [ChessDataset(BASE_PATH, "train", category) for category in CATEGORIES]


train_iter = itertools.chain.from_iterable(train_datasets)
source_vocab = build_vocab_from_iterator(
    map(lambda x: x[0].split(" "), train_iter),
    min_freq=1,
    specials=special_symbols,
    special_first=True
)

train_iter = itertools.chain.from_iterable(train_datasets)
target_vocab = build_vocab_from_iterator(
    map(lambda x: x[1].split(" "), train_iter),
    min_freq=1,
    specials=special_symbols,
    special_first=True
)

source_vocab.set_default_index(UNK_IDX)
target_vocab.set_default_index(UNK_IDX)

def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

source_transform = sequential_transforms(
    lambda x: x.split(" "),
    source_vocab,
    tensor_transform
)

target_transform = sequential_transforms(
    target_token_transform,
    target_vocab,
    tensor_transform
)

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_sample = src_sample.rstrip("\n")
        tgt_sample = tgt_sample.rstrip("\n")
        src_batch.append(source_transform(src_sample))
        tgt_batch.append(target_transform(tgt_sample))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

def get_dataloader_for(split, category, batch_size):
    return torch.utils.data.DataLoader(ChessDataset(BASE_PATH, split, category), batch_size=batch_size, collate_fn=collate_fn)


In [5]:
#model
class PositionalEncoding(torch.nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])
    
class TokenEmbedding(torch.nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
    
class Seq2SeqTransformer(torch.nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = torch.nn.Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = torch.nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: torch.Tensor,
                trg: torch.Tensor,
                src_mask: torch.Tensor,
                tgt_mask: torch.Tensor,
                src_padding_mask: torch.Tensor,
                tgt_padding_mask: torch.Tensor,
                memory_key_padding_mask: torch.Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: torch.Tensor, src_mask: torch.Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: torch.Tensor, memory: torch.Tensor, tgt_mask: torch.Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
    


In [6]:

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [7]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, dataloader, loss_fn):
    model.train()
    losses = 0
    for src, tgt in dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(dataloader))


def evaluate(model, dataloader, loss_fn):
    model.eval()
    losses = 0

    for src, tgt in dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(dataloader))

In [19]:
def train_model_for_category(category: str, transformer = None):
    NUM_EPOCHS = 200
    torch.manual_seed(0)
    SRC_VOCAB_SIZE = len(source_vocab)
    TGT_VOCAB_SIZE = len(target_vocab)
    EMB_SIZE = 64
    NHEAD = 8
    FFN_HID_DIM = 256
    BATCH_SIZE = 128
    NUM_ENCODER_LAYERS = 3
    NUM_DECODER_LAYERS = 3
    
    if transformer is None:
        transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                         NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
        
        for p in transformer.parameters():
            if p.dim() > 1:
                torch.nn.init.xavier_uniform_(p)
    
    val_dataloader = get_dataloader_for( "valid", category, BATCH_SIZE)
    dataloader = get_dataloader_for( "train", category, BATCH_SIZE)
    
    transformer = transformer.to(DEVICE)
    
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    
    optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

    
    for epoch in range(1, NUM_EPOCHS+1):
        start_time = timer()
        train_loss = train_epoch(transformer, optimizer, dataloader, loss_fn)
        end_time = timer()
        val_loss = evaluate(transformer, val_dataloader, loss_fn)
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    return transformer

In [20]:
model = train_model_for_category(CATEGORIES[-1])

/home/georgerapeanu/Desktop/BBU-Computer-Science/Semester5/Research Project/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch: 1, Train loss: 9.443, Val loss: 9.273, Epoch time = 52.036s
Epoch: 2, Train loss: 9.152, Val loss: 8.969, Epoch time = 48.488s
Epoch: 3, Train loss: 8.853, Val loss: 8.666, Epoch time = 48.823s
Epoch: 4, Train loss: 8.558, Val loss: 8.372, Epoch time = 49.503s
Epoch: 5, Train loss: 8.273, Val loss: 8.090, Epoch time = 48.783s
Epoch: 6, Train loss: 7.998, Val loss: 7.820, Epoch time = 49.187s
Epoch: 7, Train loss: 7.734, Val loss: 7.563, Epoch time = 48.860s
Epoch: 8, Train loss: 7.483, Val loss: 7.321, Epoch time = 48.682s
Epoch: 9, Train loss: 7.247, Val loss: 7.093, Epoch time = 48.773s
Epoch: 10, Train loss: 7.026, Val loss: 6.881, Epoch time = 49.835s
Epoch: 11, Train loss: 6.821, Val loss: 6.687, Epoch time = 48.622s
Epoch: 12, Train loss: 6.633, Val loss: 6.510, Epoch time = 48.760s
Epoch: 13, Train loss: 6.462, Val loss: 6.351, Epoch time = 48.812s
Epoch: 14, Train loss: 6.309, Val loss: 6.210, Epoch time = 48.897s
Epoch: 15, Train loss: 6.173, Val loss: 6.088, Epoch time

In [21]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    print(memory, ys)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = source_transform(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    print(src)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(target_vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")


In [11]:
ds = ChessDataset(BASE_PATH, "test", CATEGORIES[-1])

In [12]:
dl = get_dataloader_for("test", CATEGORIES[-1], 16)

In [13]:
for elem in dl:
    print(elem[1])
    break

tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2],
        [   31,    13,    41,   208,    28,    28,    28,   922,    31,   303,
           327,  2171,    20,     5,   187,    28],
        [   37,    32,    55,    31,   868,   336,    93,    37,    21,    40,
           873,    50,  5318,     0,    17,   253],
        [   36,  2990,   657,   154,    55,   134,  1208,    36,    12,    25,
            10,    23,     6,    40,     5,    17],
        [   45,  1295,     5,    36,    50,     5,    37,    45,  1967,   531,
           130,    40,  2289,    73,    22,     5],
        [  130,    21,    22,    45,   110,    22,    36,    12,    82,    10,
            12,    10,    14,     5,    37,    18],
        [   12,   130,  6129,    12,    21,   557,    45,   149,    29,   724,
          3570,  1109,  1819,   235,    36,     6],
        [  113,    12,    12,    17,    10,     5,    12,     4,   707,    24,
    

In [27]:
translate(model, ds[20][0])

tensor([[  2],
        [  7],
        [  7],
        [ 11],
        [ 72],
        [ 75],
        [  8],
        [  9],
        [106],
        [ 75],
        [ 10],
        [  3]])
tensor([[[-6.2782e-01,  4.0830e-01, -2.6158e+00,  2.3101e-01, -1.0214e+00,
           7.3075e-01, -5.7491e-01, -1.0021e+00, -2.2564e-01,  3.7325e-01,
           1.7065e+00,  2.1601e-01, -2.2015e+00, -7.6639e-02,  2.5875e-01,
           1.1678e-01, -2.2391e+00,  1.1143e+00,  3.9884e-01,  6.2058e-01,
          -4.9339e-01,  2.2082e-01, -1.6200e+00, -5.8521e-01, -4.4756e-01,
          -1.0529e+00, -3.1756e-01,  1.4897e+00,  8.8275e-01,  1.2393e+00,
           3.3516e-01,  1.0295e+00, -3.8907e-01,  5.1055e-01,  9.7723e-01,
          -1.2917e-02, -7.7711e-02, -7.7420e-01, -4.5730e-01, -4.4206e-01,
           7.2881e-01,  1.0394e+00, -5.1978e-01, -9.2642e-01,  4.4839e-01,
           9.0434e-01,  3.0082e-01, -5.5173e-01,  9.0435e-02,  6.0864e-01,
           4.5940e-01,  3.6979e-01,  2.2506e+00, -1.3346e+00, -1.0078

' better was to develop the knight . '

In [38]:
ds[1][0]

'black black pawn h7 h5 <EOM> <EOMH> 17... h5 <EOR> white bishop <EOPA> <EOCA>'